In [3]:
import os
import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import glob
import pickle
import matplotlib.pyplot as plt
from scipy.stats import norm
import scipy
from utils._bootstrap import bootstrap
import sympy

VAR_NAME = "D-CIPHER"
MSE_NAME = "Abl. D-CIPHER"
CONF = 0.682689 # 1 sigma

In [4]:
def combine(equation, var_or_mse):
    meta_reg = os.path.join(equation,var_or_mse,'*.p')
    meta_files = glob.glob(meta_reg)
    csv_files = [file.split('_meta.p')[0]+'_table.csv' for file in meta_files]
    dfs = []
    for meta_file, csv_file in zip(meta_files,csv_files):
        df = pd.read_csv(csv_file)
        with open(meta_file, 'rb') as f:
            setting = pickle.load(f)
            args = setting['arguments']
            gp_config = setting['gp_config']
            df['name'] = args.name
            df['filed_index'] = args.field_index
            df['width'] = args.width
            df['frequency_per_dim'] = args.frequency_per_dim
            df['noise_ratio'] = args.noise_ratio

            if var_or_mse == 'var':
                df['full_grid_samples'] = args.full_grid_samples
                df['max_ind_basis'] = args.max_ind_basis
                df['basis'] = args.basis
            elif var_or_mse == 'mse':
                df['diff_engine'] = args.diff_engine

            df['conditions_set'] = args.conditions_set
            df['num_trials'] = args.num_trials
            df['normalization'] = args.normalization
            df['solver'] = args.solver
            df['global_seed'] = args.seed
            df['num_samples'] = args.num_samples
            df['source'] = setting['table']
            for key in gp_config.keys():
                if key not in ['function_set']:
                    df[key] = gp_config[key]
        dfs.append(df)    
    full_df = pd.concat(dfs,ignore_index=True)
    full_df.drop(columns=['Unnamed: 0'],inplace=True)
    return full_df

In [5]:
def filter_df(df, filter_dict):
    mask_list = [df[key]==filter_dict[key] for key in filter_dict.keys()]
    global_mask = np.all(mask_list,axis=0)
    return df[global_mask]
    

In [6]:
# This function generates all equivalent functional forms of the given equation based on the given substitution dictionary as in Appendix E.8

X0,X1,X2,X3 = sympy.symbols('X0,X1,X2,X3',real=True)
C,C0,C1,C2,C3,C4,C5 = sympy.symbols('C,C0,C1,C2,C3,C4,C5')

import itertools
def generate_expr_list(f,sub_dict):
    keys, values = zip(*sub_dict.items())
    sub_variant_list = [dict(zip(keys, v)) for v in itertools.product(*values)]
    expr_list = []
    for sub_variant in sub_variant_list:
        g = f
        for key in sub_variant.keys():
            g = g.subs(key,sub_variant[key])
        expr_list.append(str(g))
    return expr_list


In [7]:
# Most equations are checked for correctness by the program but some may be miscategorized.
# This function allows for checking the correctness of equations according th the definition in Appendix E.8

def evaluate_correct(df, exprs, verbose=False):
    new_df = df.copy()
    for index, row in new_df.iterrows():
        truth_list = []
        eqC = row['eqC']
        for expr in exprs:
            truth_list.append(eqC == expr)
        if np.sum(truth_list) > 0:
            if (new_df.loc[index,'is_correct'] == False) and verbose:
                print(f"Changed to true: {eqC}")
            new_df.loc[index,'is_correct'] = True
        else:
            if (new_df.loc[index,'is_correct'] == True) and verbose:
                print(f"Changed to false: {eqC}")
            new_df.loc[index,'is_correct'] = False
    return new_df

In [8]:
def table_success(var_df, mse_df, by, conf, default_dict = {'noise_ratio':0.01,'num_samples':10,'delta_t':0.1, 'global_seed':2}):
    new_dict = default_dict.copy()
    new_dict.pop(by, None)
    var = filter_df(var_df,new_dict).groupby(by)['is_correct']
    mse = filter_df(mse_df,new_dict).groupby(by)['is_correct']
    z = norm.ppf(1 - (1-conf)/2)
    var_int = list(z*np.sqrt((var.mean() * (1-var.mean()))/var.count()))
    mse_int = list(z*np.sqrt((mse.mean() * (1-mse.mean()))/mse.count()))
    
    # result_df = pd.join(var.mean(),mse.mean(),on=by,suffixes=('_var','_mse'))
    # rewrite using merge
    result_df = pd.merge(var.mean(),mse.mean(),on=by,suffixes=('_var','_mse'))

    result_df['var_int'] = var_int
    result_df['mse_int'] = mse_int

    result_df.columns = ['D-CIPHER','Ablated','D-CIPHER std','Ablated std']

    result_df = result_df[['D-CIPHER','D-CIPHER std','Ablated','Ablated std']]
    
    return result_df
    
def table_operator_difference_bootstrap(var_df, mse_df, by,conf,num_operators, signs, default_dict = {'noise_ratio':0.01,'num_samples':10,'delta_t':0.1, 'global_seed':2}):
    
    new_dict = default_dict.copy()
    new_dict.pop(by, None)
    found_operator_columns = [f"operator_{i}" for i in range(num_operators)]

    target_operator_columns = [f"target_weights_{i}" for i in range(num_operators)]
    var_f = filter_df(var_df,new_dict).copy()
    mse_f = filter_df(mse_df,new_dict).copy()
    
    var_f['diff'] = 0.0
    for i in range(num_operators):
        var_f['diff'] += (var_f[f'operator_{i}'] - signs[i]*var_f[f'target_weights_{i}']) ** 2
    var_f['diff'] = np.sqrt(var_f['diff']/num_operators)
    mse_f['diff'] = 0.0
    for i in range(num_operators):
        mse_f['diff'] += (mse_f[f'operator_{i}'] - mse_f[f'target_weights_{i}']) ** 2
    mse_f['diff'] = np.sqrt(mse_f['diff']/num_operators)
    
    var = var_f
    mse = mse_f

    params = var[by].unique()
    params.sort()
    
    var_res = [bootstrap(var[var[by] == param]['diff'].to_numpy(float).reshape(1,-1),np.mean,vectorized=True,confidence_level=conf) for param in params]
    
    var_stds = [i.standard_error for i in var_res]
    var_lows = [i.confidence_interval.low for i in var_res]
    var_highs = [i.confidence_interval.high for i in var_res]
    
    var_means = var.groupby(by)['diff'].mean()
    
    mse_res = [bootstrap(mse[mse[by] == param]['diff'].to_numpy(float).reshape(1,-1),np.mean,vectorized=True,confidence_level=conf) for param in params]
    mse_stds = [i.standard_error for i in mse_res]
    mse_lows = [i.confidence_interval.low for i in mse_res]
    mse_highs = [i.confidence_interval.high for i in mse_res]
    
    mse_means = mse.groupby(by)['diff'].mean()

    result_df = pd.merge(var_means,mse_means,on=by,suffixes=('_var','_mse'))
    
    result_df['var_int'] = var_stds
    result_df['mse_int'] = mse_stds

    result_df.columns = ['D-CIPHER','Ablated','D-CIPHER std','Ablated std']
    result_df = result_df[['D-CIPHER','D-CIPHER std','Ablated','Ablated std']]
    return result_df
    # print(var_means.loc[params])
    # print(var_stds)
    
    # print(mse_means.loc[params])
    # print(mse_stds)

In [9]:
var_heat_df = combine("../results/Heat5",'var')
mse_heat_df = combine("../results/Heat5",'mse')
f = C0*sympy.exp(C1*X0+C2) + C3
sub_dict = {
    C0:[1,C],
    C1:[1,C],
    C2:[0,C,-C],
    C3:[0,C,-C],
}
expr_list = generate_expr_list(f,sub_dict)

var_heat_df = evaluate_correct(var_heat_df,expr_list)
mse_heat_df = evaluate_correct(mse_heat_df,expr_list)

print("Success probability")
print(table_success(var_heat_df, mse_heat_df,'noise_ratio',CONF,default_dict={'noise_ratio':0.01,'num_samples':10, 'global_seed':2}))

print("Average RMSE")
print(table_operator_difference_bootstrap(var_heat_df, mse_heat_df,'noise_ratio',CONF,5,[-1,-1,1,1,1],default_dict = {'noise_ratio':0.01,'num_samples':10, 'global_seed':2}))


Success probability
             D-CIPHER  D-CIPHER std  Ablated  Ablated std
noise_ratio                                              
0.05             0.64      0.067882     0.46     0.070484
0.10             0.42      0.069800     0.20     0.056568
0.20             0.12      0.045956     0.04     0.027713
Average RMSE
             D-CIPHER  D-CIPHER std   Ablated  Ablated std
noise_ratio                                               
0.05         0.152813      0.008604  0.181458     0.008725
0.10         0.208963      0.007200  0.236291     0.008001
0.20         0.244196      0.005549  0.273323     0.006535
